# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np
from datetime import datetime

In [2]:
help(glob.glob)

Help on function glob in module glob:

glob(pathname, *, recursive=False)
    Return a list of paths matching a pathname pattern.
    
    The pattern may contain simple shell-style wildcards a la
    fnmatch. However, unlike fnmatch, filenames starting with a
    dot are special cases that are not matched by '*' and '?'
    patterns.
    
    If recursive is true, the pattern '**' will match any files and
    zero or more directories and subdirectories.



In [7]:
help(os.path.abspath)

Help on function abspath in module posixpath:

abspath(path)
    Return an absolute path.



In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    
    return all_files

In [5]:
filepath = "data/song_data"
all_files=[]
#root,dirs,files = os.walk(filepath)
for root, dirs, files in os.walk(filepath):
    pattern = os.path.join(root,'*.json')
    files = glob.glob(os.path.join(root,'*.json'))
    #print(root,dirs,files)
    for f in files:
        all_files.append(os.path.abspath(f))
        #print(all_files)

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files(filepath = "data/song_data")
song_files

['/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/workspace/data/song_data/A/A/B/TRAABNV128F425CEE1.json',
 '/home/workspace/data/song_data/A/A/B/TRAABRB128F9306DD5.json',
 '/home/workspace/data/song_data/A/A/B/TRAABLR128F423B7E3.json',
 '/home/workspace/data/song_data/A/A/B/TRAABDL12903CAABBA.json',
 '/home/workspace/data/so

In [6]:
filepath = song_files[1]
filepath

'/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json'

In [7]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df.loc[0,["song_id","title","artist_id","year","duration"]]
song_data=song_data.to_dict()
song_data['year'] = int(song_data['year'])
song_data =list(song_data.values())
song_data


['SOUDSGM12AC9618304',
 'Insatiable (Instrumental Version)',
 'ARNTLGG11E2835DDB9',
 0,
 266.39627999999999]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df.loc[0,["artist_id","artist_name","artist_location","artist_latitude","artist_longitude"]].values
artist_data= artist_data.tolist()
artist_data

['ARNTLGG11E2835DDB9', 'Clp', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [10]:
log_files = get_files(filepath = "data/log_data")
log_files

['/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/lo

In [11]:
filepath = log_files[5]
filepath

'/home/workspace/data/log_data/2018/11/2018-11-20-events.json'

In [12]:
df = pd.read_json(filepath,lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Killers,Logged In,Jayden,M,32,Graves,246.80444,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Read My Mind,200,1542672042796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25
1,Tamia,Logged In,Jayden,M,33,Graves,243.09506,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Officially Missing You (Radio Version),200,1542672288796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25
2,Randy Crawford,Logged In,Jayden,M,34,Graves,270.75873,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Almaz,200,1542672531796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25
3,Frumpies,Logged In,Jayden,M,35,Graves,134.47791,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Fuck Kitty,200,1542672801796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25
4,Julia Fordham,Logged In,Jayden,M,36,Graves,279.50975,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Girlfriend,200,1542672935796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [13]:
df = df[df.page=="NextSong"]
df_2 =df.copy()


In [14]:
df_2["ts"] = pd.to_datetime(df_2["ts"],format='%Y-%m-%dT%H:%M:%SZ',infer_datetime_format=True)
df_2.head()


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,The Killers,Logged In,Jayden,M,32,Graves,246.80444,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Read My Mind,200,1970-01-01 00:25:42.672042796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25
1,Tamia,Logged In,Jayden,M,33,Graves,243.09506,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Officially Missing You (Radio Version),200,1970-01-01 00:25:42.672288796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25
2,Randy Crawford,Logged In,Jayden,M,34,Graves,270.75873,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Almaz,200,1970-01-01 00:25:42.672531796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25
3,Frumpies,Logged In,Jayden,M,35,Graves,134.47791,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Fuck Kitty,200,1970-01-01 00:25:42.672801796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25
4,Julia Fordham,Logged In,Jayden,M,36,Graves,279.50975,paid,"Marinette, WI-MI",PUT,NextSong,1.540664e+12,594,Girlfriend,200,1970-01-01 00:25:42.672935796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",25


In [22]:
t= df_2["ts"]
t_seconds=df["ts"]//10**3

In [23]:
time_data = ([t_seconds,t.dt.hour,t.dt.day,t.dt.weekofyear,t.dt.month,t.dt.year,t.dt.weekday])
column_labels = ("start_time","hour","day","week_of_year","month","year","weekday")


In [25]:
time_df = pd.DataFrame({column_labels[0]:time_data[0],column_labels[1]:time_data[1],column_labels[2]:time_data[2],\
                        column_labels[3]:time_data[3],column_labels[4]:time_data[4],\
                        column_labels[5]:time_data[5],column_labels[6]:time_data[6]})

time_df.head()

,start_time,hour,day,week_of_year,month,year,weekday
0,1542672042,0,1,1,1,1970,3
1,1542672288,0,1,1,1,1970,3
2,1542672531,0,1,1,1,1970,3
3,1542672801,0,1,1,1,1970,3
4,1542672935,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [31]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()
    

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df_2[["userId","firstName","lastName","gender","level"]]
user_df.head()

,userId,firstName,lastName,gender,level
0,25,Jayden,Graves,M,paid
1,25,Jayden,Graves,M,paid
2,25,Jayden,Graves,M,paid
3,25,Jayden,Graves,M,paid
4,25,Jayden,Graves,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [21]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert,list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables, row.length row.song,
    cur.execute(song_select, (row.song,row.artist,row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    

    # insert songplay record
    songplay_data = (index,row.ts//10**3,row.userId,row.level,songid,artistid,
                     row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [26]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.